In [11]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split as tts
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,SimpleRNN, Dense,Input
from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=1e-3, clipnorm=1.0)
# model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [12]:
word_index = imdb.get_word_index()
rev_word_index = {value:key for key,value in word_index.items()}

In [13]:
def custom_activation(x):
    return (tf.nn.tanh(x) + 1) / 2

model=load_model('simple_rnn_model.keras',custom_objects={'custom_activation':custom_activation})
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 500, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,321,985 (5.04 MB)

 Trainable params: 1,321,601 (5.04 MB)

 Non-trainable params: 384 (1.50 KB)

In [14]:
model.get_weights()

[array([[-0.06482271,  0.01527293,  0.05498945, ...,  0.06105829,
         -0.02721087,  0.07628208],
        [ 0.49040157, -0.20952989, -0.20179662, ..., -0.25418645,
          0.00997362, -0.33066148],
        [-0.00811799,  0.03655732,  0.00082704, ...,  0.02082638,
         -0.01269622,  0.02292091],
        ...,
        [-0.02965199,  0.00726493,  0.02487084, ...,  0.00601577,
          0.06514211,  0.02369049],
        [-0.03103761,  0.10342724,  0.06317542, ...,  0.12120723,
         -0.02829787,  0.06216958],
        [-0.08496086,  0.01132562,  0.01639937, ...,  0.13259946,
         -0.04544186,  0.07218817]], dtype=float32),
 array([[-0.01240041,  0.00243348, -0.0248678 , ..., -0.00044275,
         -0.0057993 ,  0.00158271],
        [ 0.01005864, -0.00362876,  0.0340967 , ...,  0.00148346,
          0.00532125,  0.00104493],
        [ 0.00600047, -0.00225304,  0.02416322, ...,  0.00043021,
          0.00657301, -0.00092466],
        ...,
        [ 0.0064407 ,  0.00266401,  0.0

In [15]:
 # Helper functions

def decode_review(encoded_review):
    return ' '.join([reverse_word_index.get(i-3,'?') for i in encoded_review])

def preprocess_text(text):
    words = text.lower().split()
    encoded_review = [min(word_index.get(word, 0), 9999) for word in words]  # 0 for OOV words
    paded_review = sequence.pad_sequences([encoded_review], maxlen=500)
    return paded_review

In [16]:
# predict sentiment

def predict_sentiment(review):
    inp = preprocess_text(review)
    prediction = model.predict(inp)
    sentiment = 'Positive' if prediction[0][0]>=0.5 else 'Negative'
    return sentiment, prediction[0][0]

In [17]:
ex1="Everything Everywhere All at Once is a genre-defying masterpiece blending sci-fi, action, and drama. Michelle Yeoh delivers a standout performance in a heartfelt story about identity and connection, enhanced by stunning visuals and inventive storytelling. A must-watch for all!"

In [18]:
sentiment,score = predict_sentiment(ex1)
print(sentiment,score)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
Positive 0.99448764


In [19]:

# Example input
ex1 = " movie is a chaotic mess that promised a mind-bending journey through multiple dimensions but delivered a disjointed narrative that left viewers bewildered rather than entertained. The film's ambitious concept falters under the weight of confusing plotlines and shallow characters, making it difficult to connect with their struggles. Despite Michelle Yeoh's talent, she is hindered by an unfocused script that demands more than is possible. The excessive use of absurd visuals, intended to illustrate the chaos of the multiverse, feels more distracting than enhancing, while the erratic pacing tests the audience's patience. Ultimately, the film is a classic case of style over substance, resulting in a forgettable experience that fails to coalesce into anything meaningful."
sentiment, score = predict_sentiment(ex1)
print(sentiment, score)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Negative 0.041336566
